In [0]:
%sql
DROP TABLE workspace.silver.search_track_artist

In [0]:
import pandas as pd
from pyspark.sql.functions import from_json, col, explode, expr, element_at
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType


# Schema para artistas
artist_schema = ArrayType(
    StructType([
        StructField("id", StringType(), True),
        StructField("name", StringType(), True)
    ])
)

# Schema para imagens do álbum
image_schema = ArrayType(
    StructType([
        StructField("height", IntegerType(), True),
        StructField("width", IntegerType(), True),
        StructField("url", StringType(), True)
    ])
)

# Schema para álbum
album_schema = StructType([
    StructField("album_type", StringType(), True),
    StructField("total_tracks", StringType(), True),
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("images", image_schema, True)
])

# Schema para track
track_schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("popularity", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("album", album_schema, True),
    StructField("artists", artist_schema, True)
])

# Lê a tabela bronze (json cru)
df = spark.table("workspace.bronze.search_track_artist")

# Faz o parse do JSON
df_parsed = df.withColumn("parsed", from_json(col("json"), track_schema))

# Seleciona colunas principais + pega a primeira imagem do álbum
df_selected = df_parsed.select(
    col("parsed.id").alias("track_id"),
    col("parsed.name").alias("track_name"),
    col("parsed.popularity"),
    col("parsed.duration_ms"),
    col("parsed.album.album_type").alias("album_type"),
    col("parsed.album.total_tracks").alias("total_tracks"),
    col("parsed.album.id").alias("album_id"),
    col("parsed.album.name").alias("album_name"),
    expr("get(parsed.album.images, 0).url").alias("album_image_url"),  # pega a 1ª imagem, mas devolve NULL se não tiver
    col("parsed.artists").alias("artists")
)

# Explodindo os artistas
df_exploded = df_selected.withColumn("artist", explode("artists")) \
    .select(
        "track_id",
        "track_name",
        "popularity",
        "duration_ms",
        "album_type",
        "total_tracks",
        "album_id",
        "album_name",
        "album_image_url",
        col("artist.id").alias("artist_id"),
        col("artist.name").alias("artist_name")
    )

# Salva como Delta Table silver
df_exploded.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.silver.search_track_artist")

In [0]:
%sql
SELECT distinct track_name FROM workspace.silver.search_track_artist
where artist_name like '%Jackson 5%'

In [0]:
%sql
SELECT distinct track, artist FROM workspace.silver.classic_hit 
where artist not in (select artist_name from workspace.silver.search_track_artist)